z-score: Used when the population standard deviation is known or when the sample size is large (typically, n≥30) even if the population standard deviation is unknown. The Z-score is based on the standard normal distribution.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats



# hypothetical data
np.random.seed(0)  
data = {
    'Brand': np.random.choice(['Apple', 'Samsung', 'Huawei', 'Xiaomi', 'Nokia'], 100),
    'Year': np.random.choice([2023], 100),
    'Price': np.random.randint(200, 1200, 100)  # Assuming prices range from 200 to 1200
}


In [2]:
df = pd.DataFrame(data)

df_2023 = df[df['Year'] == 2023]

mean_prices = df_2023.groupby('Brand')['Price'].mean()

std_devs = df_2023.groupby('Brand')['Price'].std()
sample_sizes = df_2023.groupby('Brand')['Price'].count()

# 98% confidence interval
alpha = 0.02  
conf_intervals = {}

for brand in mean_prices.index:
    mean = mean_prices[brand]
    std_dev = std_devs[brand]
    ci = stats.norm.interval(1-alpha, loc=mean, scale=std_dev)
    conf_intervals[brand] = ci


print("Average Prices and 98% Confidence Intervals for 2023:")
for brand in mean_prices.index:
    print(f"{brand}: Average Price = {mean_prices[brand]:.2f}, 98% CI = {conf_intervals[brand]}")


Average Prices and 98% Confidence Intervals for 2023:
Apple: Average Price = 660.32, 98% CI = (-82.34911154148188, 1402.9854751778457)
Huawei: Average Price = 725.82, 98% CI = (92.82536547558811, 1358.8216933479414)
Nokia: Average Price = 794.37, 98% CI = (107.90909781519781, 1480.8277442900653)
Samsung: Average Price = 592.90, 98% CI = (-134.82052994839694, 1320.6205299483968)
Xiaomi: Average Price = 600.68, 98% CI = (-95.76775691317289, 1297.1313932768091)
